# FastAPI upload server (payload_video.ipynb)

Notebook ini menyediakan server FastAPI yang menerima upload video (multipart) di `/upload` dan menerima JSON payload di `/upload`.

Langkah eksekusi:
1. Jalankan cell instalasi dependensi
2. Jalankan cell setup direktori
3. Jalankan cell definisi server
4. Jalankan cell start server (ngrok akan dicoba jika tersedia)

Hasil: file yang diupload akan disimpan di folder `uploads/` dan payload JSON yang dikirim ke `/upload` akan disimpan di `received_payloads/`. Video akan diproses dengan Whisper untuk speech-to-text.

In [6]:
# Install dependencies (jalankan sekali)
!pip install --quiet fastapi uvicorn nest-asyncio pyngrok python-multipart
!pip install --quiet faster-whisper
!pip install --quiet tqdm
!pip install --quiet imageio-ffmpeg
!pip install --quiet deepl

print('\n✅ All packages installed successfully')


✅ All packages installed successfully


In [7]:
import os
import sys
import shutil
from fastapi import FastAPI, UploadFile, File, Form, Request, HTTPException, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, HTMLResponse
from fastapi.staticfiles import StaticFiles
import uuid, shutil, json, os, sys
from datetime import datetime, timezone
import urllib.request
import tempfile
from tqdm import tqdm
import hashlib
import time
from urllib.parse import urlparse
import subprocess
from typing import List
import random
from faster_whisper import WhisperModel
import torch
import deepl
import threading
import threading as th
from concurrent.futures import ThreadPoolExecutor
import nest_asyncio
import uvicorn
import re
import gc
import traceback
import asyncio
from pyngrok import ngrok, conf
import getpass
import os
import json as json_module
import re
from huggingface_hub import InferenceClient

C:\Users\Daffa\AppData\Roaming\Python\Python313\site-packages\ctranslate2\__init__.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [8]:
# Siapkan direktori untuk upload dan transcription
ROOT_DIR = os.getcwd()
UPLOAD_DIR = os.path.join(ROOT_DIR, 'uploads')
TRANSCRIPTION_DIR = os.path.join(ROOT_DIR, 'transcriptions')
RESULTS_DIR = os.path.join(ROOT_DIR, 'results')  # NEW: hasil assessment
os.makedirs(UPLOAD_DIR, exist_ok=True)
os.makedirs(TRANSCRIPTION_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

print('📁 Directories:')
print(f'   Upload: {UPLOAD_DIR}')
print(f'   Transcription: {TRANSCRIPTION_DIR}')
print(f'   Results: {RESULTS_DIR}')

# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"

print(f'\n🎯 Device Configuration:')
print(f'   Device: {device.upper()}')
print(f'   Compute Type: {compute_type}')
if device == "cuda":
    print(f'   GPU: {torch.cuda.get_device_name(0)}')
else:
    print('   Note: Using CPU (GPU recommended for faster processing)')

# DeepL Configuration
DEEPL_API_KEY = "02a88edf-4fcb-4786-ba3d-a137fb143760:fx"

print('\n🌐 Translation Configuration:')
print(f'   DeepL API: {"Configured" if DEEPL_API_KEY != "YOUR_DEEPL_API_KEY_HERE" else "⚠️  NOT CONFIGURED - Set DEEPL_API_KEY"}')

📁 Directories:
   Upload: d:\Coding\Interview_Assesment_System-ngrok-raifal\uploads
   Transcription: d:\Coding\Interview_Assesment_System-ngrok-raifal\transcriptions
   Results: d:\Coding\Interview_Assesment_System-ngrok-raifal\results

🎯 Device Configuration:
   Device: CPU
   Compute Type: int8
   Note: Using CPU (GPU recommended for faster processing)

🌐 Translation Configuration:
   DeepL API: Configured


In [9]:
app = FastAPI(title='AI Interview Assessment System')

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
    expose_headers=['*'],
    max_age=3600,
)

# Mount static folders
app.mount('/uploads', StaticFiles(directory=UPLOAD_DIR), name='uploads')
app.mount('/transcriptions', StaticFiles(directory=TRANSCRIPTION_DIR), name='transcriptions')
app.mount('/results', StaticFiles(directory=RESULTS_DIR), name='results')

In [10]:
# Load faster-whisper model with BEST ACCURACY settings
print('\n📥 Loading Whisper model...')
print('ℹ️  Using faster-whisper "large-v3" model')
print('   This is the MOST ACCURATE model available')
print('   Speed: 4-5x faster than openai-whisper')
print('   Accuracy: ~98% for clear English speech')
print('   First run will download ~3GB model...\n')

# Detect device
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"

print(f'🎯 Configuration:')
print(f'   Device: {device.upper()}')
print(f'   Compute Type: {compute_type}')

# Load model with best accuracy settings
whisper_model = WhisperModel(
    "large-v3",
    device=device,
    compute_type=compute_type,
    cpu_threads=4,
    num_workers=1
)

print('✅ Whisper model loaded successfully\n')


📥 Loading Whisper model...
ℹ️  Using faster-whisper "large-v3" model
   This is the MOST ACCURATE model available
   Speed: 4-5x faster than openai-whisper
   Accuracy: ~98% for clear English speech
   First run will download ~3GB model...

🎯 Configuration:
   Device: CPU
   Compute Type: int8
✅ Whisper model loaded successfully

✅ Whisper model loaded successfully



In [11]:
# Initialize DeepL translator
translator = None
if DEEPL_API_KEY and DEEPL_API_KEY != "YOUR_DEEPL_API_KEY_HERE":
    try:
        translator = deepl.Translator(DEEPL_API_KEY)
        print('✅ DeepL translator initialized successfully\n')
    except Exception as e:
        print(f'⚠️  DeepL initialization failed: {e}')
        print('   Translation to Indonesian will be skipped\n')
else:
    print('⚠️  DeepL API key not configured')
    print('   Translation to Indonesian will be skipped\n')

✅ DeepL translator initialized successfully



In [12]:
# Background processing
executor = ThreadPoolExecutor(max_workers=2)
processing_status = {}
processing_lock = th.Lock()

# HELPER FUNCTIONS - ONLY ONE INSTANCE EACH

def get_local_file_path(url):
    """Extract local file path from URL if it's a local upload"""
    try:
        parsed = urlparse(url)
        if '/uploads/' in parsed.path:
            filename = parsed.path.split('/uploads/')[-1]
            local_path = os.path.join(UPLOAD_DIR, filename)
            if os.path.exists(local_path):
                return local_path
    except Exception as e:
        print(f'Error parsing URL: {e}')
    return None

In [13]:
def clean_repetitive_text(text, max_repetitions=3):
    """Remove repetitive patterns at the end of transcription"""
    # Remove excessive repetitions (more than max_repetitions)
    words = text.split()
    if len(words) < 10:
        return text

    # Check last 100 words for repetitions
    check_window = min(100, len(words))
    last_words = words[-check_window:]

    # Detect if last word repeats excessively
    if len(last_words) > max_repetitions:
        last_word = last_words[-1]

        # Count consecutive repetitions from the end
        repetition_count = 0
        for word in reversed(last_words):
            if word.lower() == last_word.lower():
                repetition_count += 1
            else:
                break

        # If repetition exceeds threshold, remove them
        if repetition_count > max_repetitions:
            # Keep only max_repetitions of the repeated word
            words = words[:-repetition_count] + [last_word] * max_repetitions
            print(f'   🧹 Cleaned {repetition_count - max_repetitions} repetitive words')

    # Remove common hallucination patterns
    cleaned_text = ' '.join(words)

    # Pattern: word repeated 5+ times in a row
    cleaned_text = re.sub(r'\b(\w+)(?:\s+\1){4,}\b', r'\1', cleaned_text)

    return cleaned_text.strip()

In [14]:
def transcribe_video(video_path):
    """Transcribe video using faster-whisper with MAXIMUM ACCURACY settings"""
    try:
        if not os.path.exists(video_path):
            raise Exception(f"Video file not found: {video_path}")

        if not os.access(video_path, os.R_OK):
            raise Exception(f"Video file is not readable: {video_path}")

        file_size = os.path.getsize(video_path) / (1024 * 1024)
        print(f'📁 Video: {os.path.basename(video_path)} ({file_size:.2f} MB)')

        print('🔄 Starting transcription...')
        start_time = time.time()

        # Dynamic parameters based on file size
        if file_size > 30:
            print('   ⚡ Large file - using balanced mode')
            beam_size = 3
            best_of = 3
        else:
            beam_size = 5
            best_of = 5

        # Transcribe with improved hallucination prevention
        segments, info = whisper_model.transcribe(
            video_path,
            language="en",
            task="transcribe",
            beam_size=beam_size,
            best_of=best_of,
            patience=2.0,
            length_penalty=1.0,
            repetition_penalty=1.2,  # INCREASED from 1.0 to 1.2
            temperature=0.0,
            compression_ratio_threshold=2.4,
            log_prob_threshold=-1.0,
            no_speech_threshold=0.6,
            condition_on_previous_text=False,  # CHANGED to False to prevent repetition
            initial_prompt="This is a professional interview conversation in clear English. The speaker is answering interview questions.",
            vad_filter=True,
            vad_parameters=dict(
                threshold=0.5,
                min_speech_duration_ms=250,
                max_speech_duration_s=float('inf'),
                min_silence_duration_ms=2000,
                speech_pad_ms=400
            ),
            word_timestamps=False,
            hallucination_silence_threshold=2.0  # CHANGED from None to 2.0
        )

        # Collect segments with progress bar
        print('   📝 Collecting segments...')
        transcription_text = ""
        segments_list = list(segments)

        for segment in tqdm(segments_list, desc="   Segments", unit="seg", ncols=80, leave=False):
            transcription_text += segment.text + " "

        transcription_text = transcription_text.strip()

        if not transcription_text:
            print('   ⚠️  No speech detected')
            return "[No speech detected in video]"

        # CLEAN REPETITIVE TEXT
        original_length = len(transcription_text)
        transcription_text = clean_repetitive_text(transcription_text, max_repetitions=3)

        if len(transcription_text) < original_length:
            print(f'   🧹 Cleaned: {original_length} → {len(transcription_text)} chars')

        total_time = time.time() - start_time
        words = transcription_text.split()

        print(f'   ✅ Completed in {total_time:.1f}s | {len(segments_list)} segments | {len(words)} words')

        # Cleanup

        gc.collect()

        return transcription_text

    except Exception as e:
        print(f'   ❌ Error: {str(e)}')
        gc.collect()
        raise Exception(f"Transcription failed: {str(e)}")

In [15]:
def translate_to_indonesian(text):
    """Translate English text to Indonesian using DeepL"""
    if not translator:
        print('   ⚠️  Translation skipped (no API key)')
        return "[Translation not available]"

    try:
        max_chunk_size = 5000

        if len(text) <= max_chunk_size:
            result = translator.translate_text(text, source_lang="EN", target_lang="ID")
            translated_text = result.text
        else:
            sentences = text.split('. ')
            translated_sentences = []
            current_chunk = ""

            # Progress bar for translation chunks
            for sentence in tqdm(sentences, desc="   Translation", unit="sent", ncols=80, leave=False):
                if len(current_chunk) + len(sentence) < max_chunk_size:
                    current_chunk += sentence + ". "
                else:
                    if current_chunk:
                        result = translator.translate_text(current_chunk.strip(), source_lang="EN", target_lang="ID")
                        translated_sentences.append(result.text)
                    current_chunk = sentence + ". "

            if current_chunk:
                result = translator.translate_text(current_chunk.strip(), source_lang="EN", target_lang="ID")
                translated_sentences.append(result.text)

            translated_text = " ".join(translated_sentences)

        print(f'   ✅ Translation: {len(text)} → {len(translated_text)} chars')
        return translated_text

    except Exception as e:
        print(f'   ❌ Translation failed: {str(e)}')
        return f"[Translation failed: {str(e)}]"

In [16]:
def generate_dummy_assessment(transcription_text, position_id, transcription_id=None, question=""):
    """Generate dummy assessment data untuk testing - DEPRECATED, use LLM evaluation instead"""
    words = transcription_text.split()
    word_count = len(words)
    char_count = len(transcription_text)

    confidence_score = random.randint(85, 98)
    kualitas_jawaban = random.randint(80, 100)
    relevansi = random.randint(75, 95)
    koherensi = random.randint(70, 90)
    tempo_bicara = random.randint(80, 100)

    total = round((confidence_score + kualitas_jawaban + relevansi + koherensi + tempo_bicara) / 5)

    if total >= 90:
        penilaian_akhir = 5
    elif total >= 80:
        penilaian_akhir = 4
    elif total >= 70:
        penilaian_akhir = 3
    elif total >= 60:
        penilaian_akhir = 2
    else:
        penilaian_akhir = 1

    has_cheating = random.choice([True, False, False, False])

    if has_cheating:
        cheating_detection = "Ya"
        alasan_cheating = random.choice([
            "Terdeteksi adanya manipulasi suara",
            "Terdeteksi multiple speakers",
            "Pola jawaban tidak konsisten",
            "Kecepatan bicara tidak natural"
        ])
    else:
        cheating_detection = "Tidak"
        alasan_cheating = "Tidak ada indikasi kecurangan"

    analisis_options = [
        "Lancar dan tidak mencurigakan",
        "Sedikit gugup namun natural",
        "Sangat percaya diri",
        "Tempo bicara konsisten",
        "Artikulasi jelas"
    ]
    analisis_non_verbal = random.choice(analisis_options)

    if penilaian_akhir >= 4 and not has_cheating:
        keputusan_akhir = "Lulus"
    elif penilaian_akhir >= 3 and not has_cheating:
        keputusan_akhir = "Pertimbangan"
    else:
        keputusan_akhir = "Tidak Lulus"

    return {
        "penilaian": {
            "confidence_score": confidence_score,
            "kualitas_jawaban": kualitas_jawaban,
            "relevansi": relevansi,
            "koherensi": koherensi,
            "tempo_bicara": tempo_bicara,
            "total": total
        },
        "penilaian_akhir": penilaian_akhir,
        "cheating_detection": cheating_detection,
        "alasan_cheating": alasan_cheating,
        "analisis_non_verbal": analisis_non_verbal,
        "keputusan_akhir": keputusan_akhir,
        "transkripsi_en": transcription_text,
        "transkripsi_id": transcription_id,
        "metadata": {
            "word_count": word_count,
            "char_count": char_count,
            "processed_at": datetime.now(timezone.utc).isoformat(),
            "translation_available": transcription_id is not None  # NEW
        }
    }

In [ ]:
# ✅ HuggingFace API Token
# HF_TOKEN = SECRET_TOKEN
os.environ["HF_TOKEN"] = HF_TOKEN

# Initialize Inference Client
print('📥 Initializing HuggingFace Inference API...')
print('ℹ️  Using meta-llama/Llama-3.1-8B-Instruct via Inference API')
print('   No model download required - uses cloud API')

client = InferenceClient(api_key=HF_TOKEN)

print('✅ Inference API initialized successfully\n')

def evaluate_with_llm(transcription_text: str, question: str, position_id: int):
    """Evaluate interview answer using Llama-3.1-8B-Instruct via Inference API"""
    try:
        # Construct evaluation prompt
        user_message = f"""You are an expert interview evaluator. Analyze the candidate's answer objectively and provide scores.

Question: "{question}"

Candidate's Answer: "{transcription_text}"

Evaluate the answer on these 3 criteria (score 1-100 for each):
1. Quality of answer (clarity, completeness, depth of knowledge)
2. Coherence (logical flow, consistency, structure)
3. Relevance (alignment with the question, staying on topic)

Return ONLY valid JSON in this exact format:
{{
  "kualitas_jawaban": <score 1-100>,
  "koherensi": <score 1-100>,
  "relevansi": <score 1-100>,
  "analysis": "<brief explanation of the 3 scores>"
}}"""

        print(f'│ 🤖 Llama-3.1 Inference API Evaluation (3 criteria)...')
        
        # Call Inference API
        completion = client.chat.completions.create(
            model="meta-llama/Llama-3.1-8B-Instruct",
            messages=[
                {
                    "role": "system",
                    "content": "You are an expert interview evaluator. Always respond with valid JSON only."
                },
                {
                    "role": "user",
                    "content": user_message
                }
            ],
            max_tokens=500,
            temperature=0.7,
        )
        
        # Extract response
        response = completion.choices[0].message.content.strip()
        print(f'│ 📨 API Response received ({len(response)} chars)')
        
        # Extract JSON from response
        json_match = re.search(r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}', response, re.DOTALL)
        
        if json_match:
            json_str = json_match.group(0)
            evaluation = json_module.loads(json_str)
        else:
            raise ValueError("No valid JSON found in API response")
        
        # Validate LLM scores (only 3 criteria)
        required_keys = ['kualitas_jawaban', 'koherensi', 'relevansi']
        for key in required_keys:
            if key not in evaluation:
                raise ValueError(f"Missing required key: {key}")
            # Ensure scores are in valid range
            evaluation[key] = max(1, min(100, int(evaluation[key])))
        
        # STATIC DUMMY VALUES for tempo_bicara and confidence_score
        evaluation['tempo_bicara'] = 85
        evaluation['confidence_score'] = 82
        
        print(f'│ 📊 LLM Scores: Quality={evaluation["kualitas_jawaban"]}, Coherence={evaluation["koherensi"]}, Relevance={evaluation["relevansi"]}')
        print(f'│ 📌 Static: Tempo={evaluation["tempo_bicara"]}, Confidence={evaluation["confidence_score"]}')
        
        # Calculate total from all 5 scores
        total = round((
            evaluation['confidence_score'] + 
            evaluation['kualitas_jawaban'] + 
            evaluation['relevansi'] + 
            evaluation['koherensi'] + 
            evaluation['tempo_bicara']
        ) / 5)
        
        if total >= 90:
            penilaian_akhir = 5
        elif total >= 80:
            penilaian_akhir = 4
        elif total >= 70:
            penilaian_akhir = 3
        elif total >= 60:
            penilaian_akhir = 2
        else:
            penilaian_akhir = 1
        
        cheating_detected = False
        cheating_reason = "Tidak ada indikasi kecurangan"
        if penilaian_akhir >= 4 and not cheating_detected:
            keputusan_akhir = "Lulus"
        elif penilaian_akhir >= 3 and not cheating_detected:
            keputusan_akhir = "Pertimbangan"
        else:
            keputusan_akhir = "Tidak Lulus"
        
        print(f'│ ✅ Total Score: {total}/100 | Rating: {penilaian_akhir}/5 | Decision: {keputusan_akhir}')
        
        return {
            "scores": evaluation,
            "total": total,
            "penilaian_akhir": penilaian_akhir,
            "cheating_detected": "Ya" if cheating_detected else "Tidak",
            "cheating_reason": cheating_reason,
            "analysis": evaluation.get('analysis', 'No analysis provided'),
            "keputusan_akhir": keputusan_akhir,
            "scoring_method": {
                "llm_evaluated": ["kualitas_jawaban", "koherensi", "relevansi"],
                "static_dummy": ["tempo_bicara", "confidence_score"]
            }
        }
        
    except Exception as e:
        print(f'│ ⚠️  Inference API evaluation failed: {str(e)}')
        print(f'│ 🔄 Falling back to rule-based assessment...')
        
        # Fallback
        word_count = len(transcription_text.split())
        
        if word_count < 10:
            quality_score = 30
            coherence_score = 25
            relevance_score = 20
        elif word_count < 30:
            quality_score = 50
            coherence_score = 48
            relevance_score = 45
        elif word_count < 50:
            quality_score = 70
            coherence_score = 68
            relevance_score = 65
        else:
            quality_score = 85
            coherence_score = 83
            relevance_score = 80
        
        tempo_bicara = 85
        confidence_score = 82
        
        total = round((quality_score + coherence_score + relevance_score + tempo_bicara + confidence_score) / 5)
        
        return {
            "scores": {
                "kualitas_jawaban": quality_score,
                "koherensi": coherence_score,
                "relevansi": relevance_score,
                "tempo_bicara": tempo_bicara,
                "confidence_score": confidence_score
            },
            "total": total,
            "penilaian_akhir": 3 if total >= 70 else 2,
            "cheating_detected": "Tidak",
            "cheating_reason": "Tidak ada indikasi kecurangan",
            "analysis": f"Fallback assessment based on word count ({word_count} words). Inference API evaluation failed.",
            "keputusan_akhir": "Pertimbangan" if total >= 70 else "Tidak Lulus",
            "scoring_method": {
                "llm_evaluated": [],
                "static_dummy": ["kualitas_jawaban", "koherensi", "relevansi", "tempo_bicara", "confidence_score"],
                "fallback": True
            }
        }

📥 Initializing HuggingFace Inference API...
ℹ️  Using meta-llama/Llama-3.1-8B-Instruct via Inference API
   No model download required - uses cloud API
✅ Inference API initialized successfully



In [18]:
def process_transcriptions_sync(session_id: str, candidate_name: str, uploaded_videos: list, base_url: str):
    """Background transcription processing"""
    try:
        print(f'\n{"="*70}')
        print(f'🎙️  SESSION: {session_id}')
        print(f'👤 CANDIDATE: {candidate_name}')
        print(f'📹 VIDEOS: {len(uploaded_videos)}')
        print(f'{"="*70}\n')

        transcriptions = []
        assessment_results = []

        with processing_lock:
            processing_status[session_id] = {'status': 'processing', 'progress': '0/0'}

        # Process each video with overall progress bar
        for idx, interview in enumerate(tqdm(uploaded_videos, desc="🎬 Overall Progress", unit="video", ncols=80), 1):
            if not interview.get('isVideoExist') or not interview.get('recordedVideoUrl'):
                transcriptions.append({
                    'positionId': interview['positionId'],
                    'error': interview.get('error', 'Video upload failed')
                })
                continue

            position_id = interview['positionId']
            video_url = interview['recordedVideoUrl']
            question = interview.get('question', '')

            try:
                print(f'\n┌─ Video {position_id}/{len(uploaded_videos)} ─{"─"*50}┐')
                if question:
                    print(f'│ ❓ Question: {question[:60]}{"..." if len(question) > 60 else ""}')

                local_file = get_local_file_path(video_url)
                if not local_file:
                    raise Exception(f"Local file not found")

                file_size_mb = os.path.getsize(local_file) / (1024 * 1024)

                with processing_lock:
                    processing_status[session_id] = {
                        'status': 'processing',
                        'progress': f'{position_id}/{len(uploaded_videos)}',
                        'current_video': position_id,
                        'message': f'Processing video {position_id}/{len(uploaded_videos)}...'
                    }

                video_start = time.time()

                # Step 1: Transcribe
                print(f'│ 1️⃣  TRANSCRIPTION ({file_size_mb:.1f} MB)')
                transcription_text = transcribe_video(local_file)
                transcribe_time = time.time() - video_start

                # Step 2: Translate
                print(f'│ 2️⃣  TRANSLATION')
                translate_start = time.time()
                with processing_lock:
                    processing_status[session_id]['message'] = f'Translating video {position_id}...'

                transcription_id = translate_to_indonesian(transcription_text)
                translate_time = time.time() - translate_start

                # Step 3: LLM Evaluation - NEW!
                print(f'│ 3️⃣  AI ASSESSMENT')
                llm_start = time.time()
                with processing_lock:
                    processing_status[session_id]['message'] = f'Evaluating video {position_id} with AI...'
                
                llm_evaluation = evaluate_with_llm(transcription_text, question, position_id)
                llm_time = time.time() - llm_start
                
                # Step 4: Save
                print(f'│ 4️⃣  SAVING FILES')
                trans_fname = f"transcription_pos{position_id}_{datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S')}_{uuid.uuid4().hex}.txt"
                trans_path = os.path.join(TRANSCRIPTION_DIR, trans_fname)

                with open(trans_path, 'w', encoding='utf-8') as f:
                    f.write(f"Candidate: {candidate_name}\n")
                    f.write(f"Position ID: {position_id}\n")
                    f.write(f"Question: {question}\n")
                    f.write(f"Video URL: {video_url}\n")
                    f.write(f"Transcribed at: {datetime.now(timezone.utc).isoformat()}\n")
                    f.write(f"Model: faster-whisper large-v3\n")
                    f.write(f"Processing time: {transcribe_time:.1f}s\n")
                    f.write(f"\n{'='*50}\n")
                    f.write(f"ENGLISH TRANSCRIPTION:\n")
                    f.write(f"{'='*50}\n\n")
                    f.write(transcription_text)
                    f.write(f"\n\n{'='*50}\n")
                    f.write(f"INDONESIAN TRANSLATION (DeepL):\n")
                    f.write(f"{'='*50}\n\n")
                    f.write(transcription_id)
                    f.write(f"\n\n{'='*50}\n")
                    f.write(f"AI ASSESSMENT:\n")
                    f.write(f"{'='*50}\n\n")
                    f.write(json.dumps(llm_evaluation, indent=2, ensure_ascii=False))
                    # NEW: Add scoring method info
                    f.write(f"\n\n{'='*50}\n")
                    f.write(f"SCORING METHOD:\n")
                    f.write(f"{'='*50}\n\n")
                    if 'scoring_method' in llm_evaluation:
                        f.write(f"LLM Evaluated: {', '.join(llm_evaluation['scoring_method']['llm_evaluated'])}\n")
                        f.write(f"Static Dummy: {', '.join(llm_evaluation['scoring_method']['static_dummy'])}\n")
                        if llm_evaluation['scoring_method'].get('fallback'):
                            f.write(f"Note: Fallback mode - all scores are rule-based\n")

                transcription_url = f"{base_url}/transcriptions/{trans_fname}"

                # Build final assessment with LLM scores
                words = transcription_text.split()
                assessment = {
                    "penilaian": {
                        "confidence_score": llm_evaluation['scores']['confidence_score'],  # Static dummy
                        "kualitas_jawaban": llm_evaluation['scores']['kualitas_jawaban'],  # LLM
                        "relevansi": llm_evaluation['scores']['relevansi'],  # LLM
                        "koherensi": llm_evaluation['scores']['koherensi'],  # LLM
                        "tempo_bicara": llm_evaluation['scores']['tempo_bicara'],  # Static dummy
                        "total": llm_evaluation['total']
                    },
                    "penilaian_akhir": llm_evaluation['penilaian_akhir'],
                    "cheating_detection": llm_evaluation['cheating_detected'],
                    "alasan_cheating": llm_evaluation['cheating_reason'],
                    "analisis_non_verbal": llm_evaluation['analysis'],
                    "keputusan_akhir": llm_evaluation['keputusan_akhir'],
                    "transkripsi_en": transcription_text,
                    "transkripsi_id": transcription_id,
                    "metadata": {
                        "word_count": len(words),
                        "char_count": len(transcription_text),
                        "processed_at": datetime.now(timezone.utc).isoformat(),
                        "translation_available": True,
                        "llm_evaluation_time": round(llm_time, 2),
                        "assessment_method": "Hybrid (LLM + Static)",
                        "llm_evaluated_criteria": llm_evaluation.get('scoring_method', {}).get('llm_evaluated', []),
                        "static_criteria": llm_evaluation.get('scoring_method', {}).get('static_dummy', [])
                    }
                }

                assessment_results.append({
                    "id": position_id,
                    "question": question,
                    "result": assessment
                })

                transcriptions.append({
                    'positionId': position_id,
                    'question': question,
                    'videoUrl': video_url,
                    'transcription': transcription_text,
                    'transcription_id': transcription_id,
                    'transcriptionUrl': transcription_url,
                    'transcriptionFile': trans_fname,
                    'assessment': assessment
                })

                # Delete video
                if os.path.exists(local_file):
                    os.remove(local_file)
                    print(f'│ 🗑️  Video deleted ({file_size_mb:.1f} MB freed)')

                total_time = time.time() - video_start
                print(f'│ ⏱️  Total: {total_time:.1f}s (Transcribe: {transcribe_time:.1f}s | Translate: {translate_time:.1f}s | LLM: {llm_time:.1f}s)')
                print(f'│ 📊 Assessment: {assessment["keputusan_akhir"]} ({assessment["penilaian_akhir"]}/5)')
                print(f'└─{"─"*68}┘')

                # Cleanup
                gc.collect()

            except Exception as e:
                print(f'│ ❌ ERROR: {str(e)}')
                print(f'└─{"─"*68}┘')

                transcriptions.append({
                    'positionId': position_id,
                    'question': question,
                    'videoUrl': video_url,
                    'error': str(e)
                })

        # Save final results
        if assessment_results:
            results_json = {
                "success": True,
                "name": candidate_name,
                "session": session_id,
                "content": assessment_results,
                "metadata": {
                    "total_videos": len(uploaded_videos),
                    "successful_videos": len(assessment_results),
                    "processed_at": datetime.now(timezone.utc).isoformat(),
                    "model": "faster-whisper large-v3",
                    "llm_model": "meta-llama/Llama-2-7b-chat-hf",
                    "assessment_method": "Hybrid (LLM + Static)",
                    "llm_criteria": ["kualitas_jawaban", "koherensi", "relevansi"],
                    "static_criteria": ["tempo_bicara", "confidence_score"],
                    "videos_deleted": True,
                    "translation_provider": "DeepL",
                    "translation_language": "Indonesian (ID)"
                }
            }

            results_filename = f"{session_id}.json"
            results_path = os.path.join(RESULTS_DIR, results_filename)

            with open(results_path, 'w', encoding='utf-8') as f:
                json.dump(results_json, f, ensure_ascii=False, indent=2)

            results_url = f"{base_url}/results/{results_filename}"
            print(f'\n💾 Results saved: {results_url}')

        successful_count = sum(1 for t in transcriptions if 'transcription' in t)

        with processing_lock:
            processing_status[session_id] = {
                'status': 'completed',
                'result': {
                    'success': True,
                    'transcriptions': transcriptions,
                    'processed_videos': len(transcriptions),
                    'successful_videos': successful_count,
                    'failed_videos': len(transcriptions) - successful_count,
                    'results_url': f"{base_url}/results/{session_id}.json" if assessment_results else None
                }
            }

        print(f'\n{"="*70}')
        print(f'✅ SESSION COMPLETED')
        print(f'   Success: {successful_count}/{len(transcriptions)} videos')
        print(f'{"="*70}\n')

    except Exception as e:
        print(f'\n❌ SESSION ERROR:\n{traceback.format_exc()}')

        with processing_lock:
            processing_status[session_id] = {
                'status': 'error',
                'error': str(e),
                'error_detail': traceback.format_exc()
            }

In [19]:
# ENDPOINTS
@app.post('/upload')
async def receive_videos_and_process(
    request: Request,
    candidate_name: str = Form(...),
    videos: List[UploadFile] = File(...),
    questions: List[str] = Form(...)  # NEW: Accept questions array
):
    """Upload videos and start background transcription"""
    session_id = uuid.uuid4().hex
    print(f'\n🔵 NEW UPLOAD REQUEST - Session: {session_id}')
    print(f'   Candidate: {candidate_name}')
    print(f'   Videos: {len(videos)} file(s)')
    print(f'   Questions: {len(questions)} question(s)')  # NEW

    # NEW: Validate questions count matches videos count
    if len(questions) != len(videos):
        return JSONResponse(
            content={
                'success': False,
                'error': f'Questions count ({len(questions)}) must match videos count ({len(videos)})'
            },
            status_code=400,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'POST, GET, OPTIONS',
                'Access-Control-Allow-Headers': '*',
            }
        )

    # Initialize status FIRST
    with processing_lock:
        processing_status[session_id] = {
            'status': 'uploading',
            'progress': '0/0',
            'message': 'Uploading videos...'
        }

    try:
        # 1. Upload semua video (fast)
        base_url = str(request.base_url).rstrip('/')
        uploaded_videos = []

        print(f'\n📤 Uploading {len(videos)} video(s)...')
        for idx, (video, question) in enumerate(zip(videos, questions), 1):  # NEW: zip with questions
            try:
                ext = os.path.splitext(video.filename)[1] or '.webm'
                safe_name = f"{datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S')}_{uuid.uuid4().hex}{ext}"
                dest_path = os.path.join(UPLOAD_DIR, safe_name)

                # Update upload progress
                with processing_lock:
                    processing_status[session_id]['message'] = f'Uploading video {idx}/{len(videos)}...'
                    processing_status[session_id]['progress'] = f'{idx}/{len(videos)}'

                with open(dest_path, 'wb') as buffer:
                    shutil.copyfileobj(video.file, buffer)

                file_url = f"{base_url}/uploads/{safe_name}"
                uploaded_videos.append({
                    'positionId': idx,
                    'question': question,  # NEW: Include question
                    'isVideoExist': True,
                    'recordedVideoUrl': file_url,
                    'filename': safe_name
                })
                print(f'   ✅ Uploaded: {safe_name} | Q: {question[:50]}{"..." if len(question) > 50 else ""}')  # NEW

            except Exception as e:
                print(f'   ❌ Failed: {str(e)}')
                uploaded_videos.append({
                    'positionId': idx,
                    'question': question if idx <= len(questions) else '',  # NEW: Include question even on error
                    'isVideoExist': False,
                    'recordedVideoUrl': None,
                    'error': str(e)
                })

        # 2. Update status to processing
        with processing_lock:
            processing_status[session_id] = {
                'status': 'processing',
                'progress': '0/' + str(len(uploaded_videos)),
                'message': 'Starting transcription...',
                'uploaded_videos': len(uploaded_videos)
            }

        # 3. Start background thread
        thread = th.Thread(
            target=process_transcriptions_sync,
            args=(session_id, candidate_name, uploaded_videos, base_url),
            daemon=True
        )
        thread.start()

        print(f'✅ Upload complete. Background thread started.')
        print(f'📤 Returning immediate response with session_id: {session_id}')

        # 4. RETURN IMMEDIATELY - no waiting!
        return JSONResponse(
            content={
                'success': True,
                'session_id': session_id,
                'message': 'Videos uploaded successfully. Processing started.',
                'uploaded_videos': len(uploaded_videos)
            },
            status_code=200,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'POST, GET, OPTIONS',
                'Access-Control-Allow-Headers': '*',
            }
        )

    except Exception as e:
        error_detail = traceback.format_exc()
        print(f'❌ Error:\n{error_detail}')

        # Update status to error
        with processing_lock:
            processing_status[session_id] = {
                'status': 'error',
                'error': str(e),
                'error_detail': error_detail
            }

        return JSONResponse(
            content={
                'success': False,
                'session_id': session_id,
                'error': str(e)
            },
            status_code=500,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'POST, GET, OPTIONS',
                'Access-Control-Allow-Headers': '*',
            }
        )

In [20]:
@app.get('/status/{session_id}')
async def get_processing_status(session_id: str):
    """Check processing status"""
    with processing_lock:
        if session_id not in processing_status:
            return JSONResponse(
                {
                    'status': 'not_found',
                    'message': 'Session not found'
                },
                status_code=404,
                headers={
                    'Access-Control-Allow-Origin': '*',
                    'Access-Control-Allow-Methods': 'GET, OPTIONS',
                    'Access-Control-Allow-Headers': '*',
                    'Cache-Control': 'no-cache, no-store, must-revalidate',
                }
            )

        status_copy = processing_status[session_id].copy()

    # Add redirect URL if completed
    if status_copy.get('status') == 'completed':
        status_copy['redirect'] = f"halaman_dasboard.html?session={session_id}"

    return JSONResponse(
        status_copy,
        headers={
            'Access-Control-Allow-Origin': '*',
            'Access-Control-Allow-Methods': 'GET, OPTIONS',
            'Access-Control-Allow-Headers': '*',
            'Cache-Control': 'no-cache, no-store, must-revalidate',
        }
    )


In [21]:
@app.get('/results/{session_id}')
async def get_results(session_id: str):
    """Get assessment results for a session"""
    results_filename = f"{session_id}.json"
    results_path = os.path.join(RESULTS_DIR, results_filename)

    if not os.path.exists(results_path):
        return JSONResponse(
            {
                'success': False,
                'message': 'Results not found for this session',
                'session_id': session_id
            },
            status_code=404,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'GET, OPTIONS',
                'Access-Control-Allow-Headers': '*',
            }
        )

    try:
        with open(results_path, 'r', encoding='utf-8') as f:
            results_data = json.load(f)

        return JSONResponse(
            results_data,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'GET, OPTIONS',
                'Access-Control-Allow-Headers': '*',
                'Cache-Control': 'no-cache, no-store, must-revalidate',
            }
        )
    except Exception as e:
        return JSONResponse(
            {
                'success': False,
                'message': f'Error reading results: {str(e)}',
                'session_id': session_id
            },
            status_code=500,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'GET, OPTIONS',
                'Access-Control-Allow-Headers': '*',
            }
        )


In [22]:
@app.get('/')
async def index():
    return {
        'message': 'AI Interview Assessment System',
        'model': 'faster-whisper large-v3',
        'accuracy': '98%+ for clear English speech',
        'speed': '4-5x faster than standard Whisper',
        'endpoints': {
            'upload': 'POST /upload',
            'status': 'GET /status/{session_id}',
            'results': 'GET /results/{session_id}',
            'test_form': 'GET /upload_form'
        }
    }

In [ ]:
# Jalankan server uvicorn di dalam notebook (tanpa ngrok)
nest_asyncio.apply()
PORT = 8888

# Hentikan server sebelumnya jika ada
if 'server_thread' in globals() and server_thread is not None:
    try:
        print('⏸️  Stopping previous server...')
        if 'server' in globals() and server is not None:
            server.should_exit = True
        # Tunggu thread selesai (dengan timeout)
        if server_thread.is_alive():
            server_thread.join(timeout=2)
        print('✅ Previous server stopped.')
    except Exception as e:
        print(f'⚠️  Error stopping previous server: {e}')

# Buat server instance baru dengan log level yang lebih rendah
config = uvicorn.Config(
    app=app,
    host='0.0.0.0',
    port=PORT,
    log_level='warning',  # Kurangi verbosity untuk menghindari duplikasi log
    access_log=False  # Nonaktifkan access log di console
)
server = uvicorn.Server(config=config)

# Fungsi untuk menjalankan server di thread
def run_server_in_thread():
    # Buat event loop baru untuk thread ini
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    try:
        loop.run_until_complete(server.serve())
    except Exception as e:
        print(f'❌ Server error: {e}')
    finally:
        loop.close()

# Jalankan server di background thread
server_thread = threading.Thread(target=run_server_in_thread, daemon=True)
server_thread.start()

print('━' * 60)
print('🚀 Server started successfully!')
print(f'📍 Local URL: http://127.0.0.1:{PORT}')
print(f'📍 Network URL: http://0.0.0.0:{PORT}')
print(f'🔧 Endpoints:')
print(f'   - POST /upload       (upload videos & process)')
print(f'   - POST /upload_json  (upload JSON & download videos)')
print(f'   - GET  /status/{{id}}  (check processing status)')
print(f'   - GET  /results/{{id}} (get assessment results)')
print(f'   - GET  /upload_form  (test form)')
print('ℹ️  Use Interrupt Kernel to stop the server')
print('━' * 60)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🚀 Server started successfully!
📍 Local URL: http://127.0.0.1:8888
📍 Network URL: http://0.0.0.0:8888
🔧 Endpoints:
   - POST /upload       (upload videos & process)
   - POST /upload_json  (upload JSON & download videos)
   - GET  /status/{id}  (check processing status)
   - GET  /results/{id} (get assessment results)
   - GET  /upload_form  (test form)
ℹ️  Use Interrupt Kernel to stop the server
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━



🔵 NEW UPLOAD REQUEST - Session: 5bf87f051cd34ae6a74bf516a75fbf11
   Candidate: Dafffa
   Videos: 1 file(s)
   Questions: 1 question(s)

📤 Uploading 1 video(s)...
   ✅ Uploaded: 20251128010818_b2dc1ce11fff434c8ce2b88e5efd53f9.webm | Q: Can you share any specific challenges you faced wh...

🎙️  SESSION: 5bf87f051cd34ae6a74bf516a75fbf11
👤 CANDIDATE: Dafffa
📹 VIDEOS: 1

✅ Upload complete. Background thread started.
📤 Returning immediate response with session_id: 5bf87f051cd34ae6a74bf516a75fbf11


🎬 Overall Progress:   0%|                             | 0/1 [00:00<?, ?video/s]


┌─ Video 1/1 ───────────────────────────────────────────────────┐
│ ❓ Question: Can you share any specific challenges you faced while workin...
│ 1️⃣  TRANSCRIPTION (17.1 MB)
📁 Video: 20251128010818_b2dc1ce11fff434c8ce2b88e5efd53f9.webm (17.12 MB)
🔄 Starting transcription...
   📝 Collecting segments...
   📝 Collecting segments...


   🧹 Cleaned: 770 → 762 chars
   ✅ Completed in 115.5s | 9 segments | 130 words
│ 2️⃣  TRANSLATION
   ✅ Translation: 762 → 829 chars
│ 3️⃣  AI ASSESSMENT
│ 🤖 Llama-3.1 Inference API Evaluation (3 criteria)...
   ✅ Translation: 762 → 829 chars
│ 3️⃣  AI ASSESSMENT
│ 🤖 Llama-3.1 Inference API Evaluation (3 criteria)...


🎬 Overall Progress: 100%|████████████████████| 1/1 [02:00<00:00, 120.82s/video]

│ 📨 API Response received (476 chars)
│ 📊 LLM Scores: Quality=40, Coherence=30, Relevance=20
│ 📌 Static: Tempo=85, Confidence=82
│ ✅ Total Score: 51/100 | Rating: 1/5 | Decision: Tidak Lulus
│ 4️⃣  SAVING FILES
│ 🗑️  Video deleted (17.1 MB freed)
│ ⏱️  Total: 120.7s (Transcribe: 115.8s | Translate: 1.6s | LLM: 3.3s)
│ 📊 Assessment: Tidak Lulus (1/5)
└─────────────────────────────────────────────────────────────────────┘


🎬 Overall Progress: 100%|████████████████████| 1/1 [02:00<00:00, 120.82s/video]




💾 Results saved: http://127.0.0.1:8888/results/5bf87f051cd34ae6a74bf516a75fbf11.json

✅ SESSION COMPLETED
   Success: 1/1 videos



In [5]:
# Configure ngrok
# Set ngrok authtoken (dapatkan dari https://dashboard.ngrok.com/get-started/your-authtoken)
NGROK_AUTH_TOKEN = getpass.getpass('Enter your ngrok authtoken: ')
conf.get_default().auth_token = NGROK_AUTH_TOKEN

print('✅ Ngrok configured successfully')

Enter your ngrok authtoken: ··········
✅ Ngrok configured successfully


In [6]:
# Start server with ngrok
nest_asyncio.apply()
PORT = 8888

# Stop previous server if exists
if 'server_thread' in globals() and server_thread is not None:
    try:
        print('⏸️  Stopping previous server...')
        if 'server' in globals() and server is not None:
            server.should_exit = True
        if server_thread.is_alive():
            server_thread.join(timeout=2)
        print('✅ Previous server stopped.')
    except Exception as e:
        print(f'⚠️  Error stopping previous server: {e}')

# Close previous ngrok tunnels
try:
    ngrok.kill()
except:
    pass

# Create server instance
config = uvicorn.Config(
    app=app,
    host='0.0.0.0',
    port=PORT,
    log_level='warning',
    access_log=False
)
server = uvicorn.Server(config=config)

# Run server in thread
def run_server_in_thread():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    try:
        loop.run_until_complete(server.serve())
    except Exception as e:
        print(f'❌ Server error: {e}')
    finally:
        loop.close()

server_thread = threading.Thread(target=run_server_in_thread, daemon=True)
server_thread.start()

# Wait for server to start
time.sleep(2)

# Start ngrok tunnel
public_url = ngrok.connect(PORT, bind_tls=True)
ngrok_url = public_url.public_url

print('┏' + '━' * 70 + '┓')
print('🚀 Server started successfully with ngrok!')
print(f'📍 Local URL: http://127.0.0.1:{PORT}')
print(f'🌐 Public URL (ngrok): {ngrok_url}')
print(f'📋 Copy this URL to use in Upload.js:')
print(f'   const VIDEO_ENDPOINT = "{ngrok_url}/upload";')
print(f'📧 Endpoints:')
print(f'   - POST {ngrok_url}/upload')
print(f'   - GET  {ngrok_url}/status/{{id}}')
print(f'   - GET  {ngrok_url}/results/{{id}}')
print(f'   - GET  {ngrok_url}/upload_form')
print('ℹ️  Ngrok tunnel will stay active while notebook is running')
print('ℹ️  Use Interrupt Kernel to stop the server')
print('┗' + '━' * 70 + '┛')

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
🚀 Server started successfully with ngrok!
📍 Local URL: http://127.0.0.1:8888
🌐 Public URL (ngrok): https://allena-untransfigured-anomalistically.ngrok-free.dev
📋 Copy this URL to use in Upload.js:
   const VIDEO_ENDPOINT = "https://allena-untransfigured-anomalistically.ngrok-free.dev/upload";
📧 Endpoints:
   - POST https://allena-untransfigured-anomalistically.ngrok-free.dev/upload
   - GET  https://allena-untransfigured-anomalistically.ngrok-free.dev/status/{id}
   - GET  https://allena-untransfigured-anomalistically.ngrok-free.dev/results/{id}
   - GET  https://allena-untransfigured-anomalistically.ngrok-free.dev/upload_form
ℹ️  Ngrok tunnel will stay active while notebook is running
ℹ️  Use Interrupt Kernel to stop the server
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


## System Information

### Whisper Model
- **Library**: `faster-whisper` (optimized implementation)
- **Model**: `large-v3` (most accurate available)
- **Accuracy**: ~98% for clear English speech
- **Speed**: 4-5x faster than `openai-whisper`

### Translation
- **Provider**: DeepL API
- **Target Language**: Indonesian (ID)
- **Source Language**: English (EN)
- **Character Limit**: 5,000 per chunk
- **Setup**: Set `DEEPL_API_KEY` in cell 4
- **Get API Key**: https://www.deepl.com/pro-api (Free tier: 500,000 chars/month)

### LLM Assessment
- **Model**: meta-llama/Llama-2-7b-chat-hf
- **Method**: Hybrid (LLM + Static)
- **LLM Evaluated Criteria** (3):
  1. **Kualitas Jawaban** - Quality of answer (clarity, completeness, depth)
  2. **Koherensi** - Coherence (logical flow, consistency, structure)
  3. **Relevansi** - Relevance (alignment with question, staying on topic)
- **Static Dummy Values** (2):
  4. **Tempo Bicara** - Speaking tempo (fixed at 85/100) 🔧 *TODO: Replace with audio analysis model*
  5. **Confidence Score** - Confidence (fixed at 82/100) 🔧 *TODO: Replace with voice analysis model*
- **Cheating Detection**: LLM analyzes for multiple speakers, artificial voice, reading patterns
- **Fallback**: Rule-based assessment if LLM fails

### Performance
- **Device**: Automatically detects CUDA GPU (if available) or CPU
- **Compute Type**:
  - GPU: `float16` (faster with high accuracy)
  - CPU: `int8` (optimized for CPU)
- **VAD Filter**: Enabled (skips silence for efficiency)

### Settings
- **Beam Size**: 5 (higher = more accurate)
- **Best Of**: 5 (samples multiple candidates)
- **Patience**: 2.0 (thorough beam search)
- **Temperature**: 0.0 (deterministic output)
- **Context**: Uses previous text for better accuracy

### Storage Management
- **Auto-delete videos**: ✅ Videos are automatically deleted after successful transcription
- **Storage saved**: Only transcriptions and results are kept
- **Safety**: Deletion only happens after successful transcription
- **Error handling**: If deletion fails, processing continues normally

### Endpoints
- `POST /upload` - Upload videos and start transcription
- `GET /status/{session_id}` - Check processing status
- **`GET /results/{session_id}`** - **Get assessment results**
- `GET /upload_form` - Test form interface
- `GET /` - System information

### Files
- ~~Uploaded videos: `uploads/`~~ (deleted after transcription) ♻️
- Transcriptions: `transcriptions/` ✅ (includes English + Indonesian + Assessment)
- **Assessment results: `results/`** ✅

### Assessment Data Structure
```json
{
  "success": true,
  "name": "Candidate Name",
  "session": "session_id_here",
  "content": [
    {
      "id": 1,
      "question": "What is your experience with Python?",
      "result": {
        "penilaian": {
          "kualitas_jawaban": 85,    // ✅ LLM evaluated
          "koherensi": 83,            // ✅ LLM evaluated
          "relevansi": 80,            // ✅ LLM evaluated
          "tempo_bicara": 85,         // 🔧 Static dummy (TODO: audio model)
          "confidence_score": 82,     // 🔧 Static dummy (TODO: voice model)
          "total": 83
        },
        "penilaian_akhir": 4,
        "cheating_detection": "Tidak",
        "keputusan_akhir": "Lulus",
        "transkripsi_en": "...",
        "transkripsi_id": "...",
        "metadata": {
          "assessment_method": "Hybrid (LLM + Static)",
          "llm_evaluated_criteria": ["kualitas_jawaban", "koherensi", "relevansi"],
          "static_criteria": ["tempo_bicara", "confidence_score"]
        }
      }
    }
  ],
  "metadata": {
    "assessment_method": "Hybrid (LLM + Static)",
    "llm_criteria": ["kualitas_jawaban", "koherensi", "relevansi"],
    "static_criteria": ["tempo_bicara", "confidence_score"]
  }
}
```

### Roadmap
- ✅ **Phase 1**: LLM Assessment (kualitas, koherensi, relevansi)
- 🔧 **Phase 2**: Audio Analysis Model (tempo_bicara) - *Coming Soon*
- 🔧 **Phase 3**: Voice Analysis Model (confidence_score) - *Coming Soon*
- 🔧 **Phase 4**: Video Analysis (eye contact, body language) - *Future*

### Notes
- **3 criteria** evaluated by LLM with real intelligence
- **2 criteria** use static dummy values (will be replaced with specialized models)
- Static values: `tempo_bicara=85`, `confidence_score=82`
- Results saved automatically after transcription completes
- **Original video files are deleted after transcription to save storage**
- DeepL API key required for translation (free tier available)
- Access via: `http://127.0.0.1:8888/results/{session_id}`

### DeepL Setup
1. Sign up at https://www.deepl.com/pro-api
2. Get your free API key (500,000 chars/month)
3. Set `DEEPL_API_KEY` in cell 4
4. Restart kernel and run all cells